In [16]:
# ====================================
# 📌 RAG Project: PDF Course Catalog + Smart Q&A
# ====================================

# ---- Install Packages ----
!pip install -q langchain langchain-community langchain-pinecone langchain-google-genai tavily-python pypdf sentence-transformers

import os
from google.colab import userdata, files

# LangChain utilities
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# LLM + Embeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceEmbeddings

In [17]:


# Vector store (Pinecone)
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

# PDF loader
from langchain_community.document_loaders import PyPDFLoader

# Tavily search
from langchain_community.tools.tavily_search import TavilySearchResults


# ====================================
# 🔑 Load API Keys
# ====================================
GOOGLE_KEY = userdata.get("GOOGLE_API_KEY")
PINECONE_KEY = userdata.get("PINECONE_API_KEY")
TAVILY_KEY = userdata.get("TAVILY_API_KEY")

os.environ["GOOGLE_API_KEY"] = GOOGLE_KEY or ""
os.environ["PINECONE_API_KEY"] = PINECONE_KEY or ""
os.environ["TAVILY_API_KEY"] = TAVILY_KEY or ""

print("🔑 Keys Loaded:", {
    "Gemini": bool(GOOGLE_KEY),
    "Pinecone": bool(PINECONE_KEY),
    "Tavily": bool(TAVILY_KEY)
})


# ====================================
# 📄 Upload & Process PDF
# ====================================
uploaded_file = files.upload()
pdf_file = list(uploaded_file.keys())[0]

loader = PyPDFLoader(pdf_file)
pages = loader.load()
print(f"📄 Loaded {len(pages)} pages from {pdf_file}")

# Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=900, chunk_overlap=150)
chunks = splitter.split_documents(pages)
print(f"✂️ Split into {len(chunks)} chunks")


# ====================================
# 🧠 Embeddings + Pinecone Setup
# ====================================
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

pc = Pinecone(api_key=PINECONE_KEY)
INDEX_NAME = "rag-course-index"

# Create index if not exists
if INDEX_NAME not in [x["name"] for x in pc.list_indexes()]:
    pc.create_index(
        name=INDEX_NAME,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print(f"✅ Created Pinecone index: {INDEX_NAME}")

vector_store = PineconeVectorStore.from_documents(chunks, embedding=emb, index_name=INDEX_NAME)
print("stored in Pinecone successfully")


# ====================================
# ⚡ Initialize LLM + Prompt
# ====================================
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0, google_api_key=GOOGLE_KEY)

qa_template = """
You are a course assistant. Use the given context to answer questions.
If the answer is missing, reply: "I don't know."

Context:
{context}

Question:
{question}

Answer:
"""
prompt = PromptTemplate(input_variables=["context", "question"], template=qa_template)
qa_chain = LLMChain(llm=llm, prompt=prompt)

# Tavily search tool
web_search = TavilySearchResults()


# ====================================
# 🤖 Smart Q&A Function
# ====================================
def query_rag(question: str, top_k: int = 3, threshold: float = 0.2):
    """Try Pinecone first, fallback to Tavily if weak results."""
    matches = vector_store.similarity_search_with_score(question, k=top_k)

    if matches and matches[0][1] >= threshold:
        context_text = "\n\n".join([m[0].page_content for m in matches])
        print(f"Answered from RAG | score={matches[0][1]:.2f}")
        return qa_chain.invoke({"context": context_text, "question": question})["text"]

    print("Tavily Web Search")
    web_results = web_search.invoke({"query": question})

    summarize_prompt = f"""
    Summarize the following search results into a clear answer:

    Question: {question}

    Results:
    {web_results}
    """
    return llm.invoke(summarize_prompt).content


# ====================================
# 🧪 Example Queries
# ====================================
print(query_rag("List all courses"))
print(query_rag("What is new in Python 3.12?"))


🔑 Keys Loaded: {'Gemini': True, 'Pinecone': True, 'Tavily': True}


Saving Course Catalog.pdf to Course Catalog (5).pdf
📄 Loaded 1 pages from Course Catalog (5).pdf
✂️ Split into 1 chunks
stored in Pinecone successfully
Answered from RAG | score=0.39
Here are all the courses:
1. Introduction to Computer Science
2. Data Structures & Algorithms
3. Database Systems
4. Operating Systems
5. Software Engineering
6. Artificial Intelligence
7. Computer Networks
8. Web Development
9. Mobile App Development
10. Capstone Project
Tavily Web Search
Python 3.12 introduces several significant new features and improvements, focusing on type hinting, performance, and developer experience:

*   **Enhanced Type Hinting:**
    *   **New type annotation syntax for generic classes (PEP 695):** Simplifies the creation of generic classes and functions.
    *   **Support for the buffer protocol (PEP 688):** Allows Python code to know if an object supports the buffer protocol and to type-annotate code for compatibility.
    *   **`TypedDict` for keyword arguments:** You can now